Communication plan based on lab report

In [ ]:
pip install google-genai python-dotenv

In [3]:
import os
from dotenv import load_dotenv
from datetime import datetime

from google import genai
from google.genai import types

In [ ]:
load_dotenv()
API_KEY = os.getenv("GEMINI_API_KEY")

client = genai.Client(api_key=API_KEY)

In [5]:
def upload_file(filepath):
    uploaded_file = client.files.upload(file=filepath)
    print(f"File uploaded successfully: {uploaded_file}")
    return uploaded_file

In [33]:
file_path = "./reports/Mr. Samuel Joglekar_lab.pdf"
uploaded_file = upload_file(file_path)
print(uploaded_file)

File uploaded successfully: name='files/xzl1r5cvkwsj' display_name=None mime_type='application/pdf' size_bytes=148211 create_time=datetime.datetime(2025, 6, 16, 11, 2, 50, 46636, tzinfo=TzInfo(UTC)) expiration_time=datetime.datetime(2025, 6, 18, 11, 2, 50, 2060, tzinfo=TzInfo(UTC)) update_time=datetime.datetime(2025, 6, 16, 11, 2, 50, 46636, tzinfo=TzInfo(UTC)) sha256_hash='NWY4MzU5OTdkZDlhMTQ0ZTY3NGQ0ZDA0YzNiNzcxYmJjOTE1MjA3NWRlZDUxYmNhZDdiY2EzMjhjMWQ2NWFjMA==' uri='https://generativelanguage.googleapis.com/v1beta/files/xzl1r5cvkwsj' download_uri=None state=<FileState.ACTIVE: 'ACTIVE'> source=<FileSource.UPLOADED: 'UPLOADED'> video_metadata=None error=None
name='files/xzl1r5cvkwsj' display_name=None mime_type='application/pdf' size_bytes=148211 create_time=datetime.datetime(2025, 6, 16, 11, 2, 50, 46636, tzinfo=TzInfo(UTC)) expiration_time=datetime.datetime(2025, 6, 18, 11, 2, 50, 2060, tzinfo=TzInfo(UTC)) update_time=datetime.datetime(2025, 6, 16, 11, 2, 50, 46636, tzinfo=TzInfo(UTC)

In [36]:
def load_prompt(prompt_path):
    with open(prompt_path, "r", encoding="utf-8") as f:
        return f.read()

system_prompt_path = "system_prompt.txt"
user_prompt_path = "user_prompt.txt"
system_prompt = load_prompt(system_prompt_path)
user_prompt = load_prompt(user_prompt_path)


In [37]:
full_prompt = system_prompt + "\n" + user_prompt

contents = [
    types.Content(
        role="user",
        parts=[
            types.Part(text=full_prompt),
            types.Part(file_data=types.FileData(file_uri=uploaded_file.uri))
        ]
    )
]



In [38]:
# Generate response
response = client.models.generate_content(
    model="models/gemini-2.5-flash-preview-05-20",
    contents=contents,
    config=types.GenerateContentConfig(
        temperature=0.3,
        max_output_tokens=20000,
    )
)
generated_text = response.text


In [12]:
import re

def remove_asterisks(text):
    # Remove all double asterisks
    return re.sub(r'\*\*(.*?)\*\*', r'\1', text)


In [39]:
# Save full interaction to log file
def save_log(response, log_path="gemini_2.5_healthcare_log3.txt"):
    response = remove_asterisks(response)
    with open(log_path, "a", encoding="utf-8") as f:
        f.write("\n\n" + response)

save_log(generated_text)


In [40]:
# delete file after processing
def delete_file(file):
    client.files.delete(name=file.name)
    print(f"File deleted successfully: {file.name}")

# Now delete using uploaded_file.id, not name:
delete_file(uploaded_file)


File deleted successfully: files/xzl1r5cvkwsj
